In [2]:
import torch
import torchvision.transforms as T
from PIL import Image
import numpy as np
from timm.models.vision_transformer import Attention
import cv2
import os
import argparse

class GradientRollout:
    def __init__(self):
        self.model = torch.hub.load('facebookresearch/deit:main', 'deit_tiny_patch16_224', pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = True
        self.attentions = []
        self.attention_grads = []
        self.num_heads = 3
        self.transform = T.Compose([
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize(
                mean=(0.485, 0.456, 0.406),  # ImageNet mean
                std=(0.229, 0.224, 0.225)    # ImageNet std
            )
        ])
        # Replace the 'forward' method of all 'Attention' modules 
        for name, module in self.model.named_modules():
            if isinstance(module, Attention):
                # Bind the attention_forward method to the module instance
                module.forward = self.attention_forward.__get__(module, Attention)

                # Register the forward hook to extract attention weights
                module.register_forward_hook(self.get_attention)

    @staticmethod
    def attention_forward(self, x):
        """

        TODO:
            Implement the attention computation and store the attention maps.
            You need to save the attention map into variable "self.attn_weights"

            Note: Due to @staticmethod, "self" here refers to the "Attention" module instance, not the class itself.

        """
        

        # write your code here
        self.attn_weights = None # save the attention map into this variable
        pass
        batch_size, seq_len, _ = x.size()
        qkv = self.qkv(x).reshape(batch_size, seq_len, 3, self.num_heads, -1).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        attn = (q @ k.transpose(-2, -1)) * (self.scale)
        attn = attn.softmax(dim=-1)
        self.attn_weights = attn  # Save the attention map
        return (attn @ v).transpose(1, 2).reshape(batch_size, seq_len, -1)

    # Define a hook function to extract attention weights
    def get_attention(self, module, input, output):
        # Append the attention weights
        self.attentions.append(module.attn_weights.detach().cpu())

        # Retain gradients on attention weights
        module.attn_weights.retain_grad()

        # Register a hook on attn_weights to save the gradients during backward pass
        def save_attn_grad(grad):
            self.attention_grads.append(grad.cpu())
        module.attn_weights.register_hook(save_attn_grad)

    def clear_attentions(self):
        # clear the stored attention weights
        self.attentions = []
        self.attention_grads = []
    
    
    def gradient_rollout(self, discard_ratio = 0.9):
        """

        TODO:
            Define the attention rollout function that accumulate the final attention flows.
            You need to return parameter "mask", which should be the final attention flows.  
            You can follow the below procedures:

            For each attention layers:
                1. perform matrix mutiplication on current attention map and gradient
                2. filter the attention by mean/min/max fiter with respect to 3 attention heads
                3. wipe out all negative values to 0
                4. (optional) normalize the attention map of current layer
                5. perform matrix multiplication on current attention map and previous results

            (6. and 7. are already done for you, you just need to get the variable "result" correctly.)
            6. Obtain the mask: Gather the attention flows that use [CLS] token as query and the rest tokens as keys
            7. Normalize the values inside the mask to [0.0, 1.0]

        """
        result = torch.eye(self.attentions[0].size(-1))
        
        for attention, grad in zip(self.attentions, self.attention_grads): 
            # Write your code(Gradient Rollout) here to update "result" matrix
            # 使用梯度作为注意力的权重
                weights = grad
                attention_heads_fused = (attention * weights).mean(dim=1)

                # 将小于0的注意力值设为0（因为负值可能没有意义）
                attention_heads_fused[attention_heads_fused < 0] = 0

                # 丢弃最小的注意力值，但保留 [CLS] token
                flat = attention_heads_fused.view(attention_heads_fused.size(0), -1)
                _, indices = flat.topk(int(flat.size(-1) * discard_ratio), dim=-1, largest=False)
                flat[0, indices] = 0

                # 添加单位矩阵用于保持原始的自注意力特性
                I = torch.eye(attention_heads_fused.size(-1), device=attention_heads_fused.device)
                a = (attention_heads_fused + I) / 2
                a = a / a.sum(dim=-1, keepdim=True)

                # 逐层累积注意力
                result = torch.matmul(a, result)
        



        """
        if you have variable "result" in correct shape(shape of attenion map), then this part should work properly.
        """
        # Look at the total attention between the class token,
        # and the image patches
        mask = result[0, 0 , 1 :]
        # In case of 224x224 image, this brings us from 196 to 14
        width = int(mask.size(-1)**0.5)
        mask = mask.reshape(width, width).numpy()
        mask = mask / np.max(mask)
        return mask


    def perform_backpropagation(self, input_tensor, target_category):
        """

        TODO:
            In order to get gradients, you need to do backpropagation mannually.
            You can follow the below procedures:

            1.Inference the model
            2.define a loss simple function that focus on the target category
            2-1. The loss function can be sum of the logits of target category or
            2-2. cross entropy of target category
            3.perform backpropagation with respect to the loss.

            This function will be invoked in "run" function.

        """

        # write your code here to perform backpropagation
        pass
        self.model.zero_grad()
        
        # 前向传播并计算输出
        output = self.model(input_tensor)
        
         # 创建类别掩码，确保仅对目标类别进行反向传播
        category_mask = torch.zeros_like(output)
        category_mask[:, target_category] = 1
        
        # 计算损失，保留目标类别的损失
        loss = (output * category_mask).sum()
    

        # 反向傳播計算梯度
        loss.backward(retain_graph=True)
        
    def show_mask_on_image(self, img, mask):

        """Do not modify this part"""

        # Normalize the value of img to [0.0, 1.0]
        img = np.float32(img) / 255

        # Reshape the mask to 224x224 for later computation
        mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

        # Generate heatmap and normalize the value
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255

        # Add heatmap and original image together and normalize the value
        combination = heatmap + np.float32(img)
        combination = combination / np.max(combination)
        
        # Scale back the value to [0.0, 255.0]
        combination = np.uint8(255 * combination)

        return combination
    
    def run(self, image_path, target_category):

        """Do not modify this part"""

        # clean previous attention maps and result
        self.clear_attentions()
        
        # get the image name for saving output image2
        image_name = os.path.basename(image_path)  # e.g., 'image.jpg'
        image_name, _ = os.path.splitext(image_name)  # ('image', '.jpg')
        
        # convert image to a tensor
        img = Image.open(image_path).convert('RGB')
        input_tensor = self.transform(img).unsqueeze(0)  # Add batch dimension
        input_tensor.requires_grad_(True)

        # do backpropagation manually during inference to produce gradients
        self.perform_backpropagation(input_tensor, target_category)

        np_img = np.array(img)[:, :, ::-1]
        mask = self.gradient_rollout()
        output_heatmap = self.show_mask_on_image(np_img, mask)
        output_filename = f"gradient_result_{image_name}.png"  
        cv2.imwrite(output_filename, output_heatmap)


# if __name__ == '__main__':

#     """Do not modify this part"""

#     # arg parsing
#     parser = argparse.ArgumentParser(description='Process an image for attention visualization with respect to target category.')
#     parser.add_argument('--image', type=str, required=True, help='Path to the input image.')
#     parser.add_argument('--category', type=int, required=True, help="target category of attention flows")
#     args = parser.parse_args()

#     # Execution
#     model = GradientRollout()
#     outputs = model.run(args.image, args.category)

In [3]:
from PIL import Image
import torch

# 初始化 GradientRollout 類
model = GradientRollout()

# 測試圖片和目標類別
image_path = "./images/part2/87_dogbird.png"
target_category = 161

# 加載圖片，轉換為張量
img = Image.open(image_path).convert('RGB')
input_tensor = model.transform(img).unsqueeze(0)  # 添加 batch 維度
input_tensor.requires_grad_(True)

# 執行 perform_backpropagation
model.perform_backpropagation(input_tensor, target_category)

# 確認輸出和損失
with torch.no_grad():
    output = model.model(input_tensor)  # 前向傳播輸出
    target_logits = output[:, target_category]
    loss = target_logits.sum()

# 打印輸出和損失
print(f"Model output shape: {output.shape}")
print(f"Target logits for category {target_category}: {target_logits.item()}")
print(f"Loss value: {loss.item()}")

# 檢查梯度
print(f"Gradients for input tensor (sum): {input_tensor.grad.sum()}")


Using cache found in C:\Users\User/.cache\torch\hub\facebookresearch_deit_main
c:\Users\User\anaconda3\envs\DeepLearning\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\User\anaconda3\envs\DeepLearning\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\User/.cache\torch\hub\facebookresearch_deit_main\models.py:62: UserWarning: Overwriting deit_tiny_patch16_224 in registry with models.deit_tiny_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
C:\Users\User/.cache\torch\hub\facebookr

RuntimeError: can't retain_grad on Tensor that has requires_grad=False